In [1]:
import pandas as pd
import numpy as np
from surprise import KNNWithMeans
from surprise import Dataset, Reader
from surprise.model_selection import KFold, cross_validate
from tabulate import tabulate
from collections import defaultdict

# Part 1: Using Surprise

## Read and load the data using Surprise

In [2]:
# Read both the datasets from the files using pandas
movielens_df = pd.read_csv("../data/u.data", sep="\t", header=None)
movielens_df.columns = ["userID", "itemID", "rating", "timestamp"]
pda_df = pd.read_csv("../data/train-PDA2018.csv", sep=",")
print("ML100K\n", movielens_df.head())
print("\n\n")
print("PDA2018\n", pda_df.head())

# Sample the data such that every user has rated at least 10 items and every item has been by at least 10 users
print("Shapes before sub-sampling:")
print(movielens_df.shape)
print(pda_df.shape)

# Movielens users all have at least 20 ratings so no need to subsample the user values
ml_subsampled = movielens_df[movielens_df['itemID'].isin(movielens_df['itemID'].value_counts()[movielens_df['itemID'].value_counts()>10].index)]
pda_subsampled = pda_df[pda_df['itemID'].isin(pda_df['itemID'].value_counts()[pda_df['itemID'].value_counts()>10].index)]
pda_subsampled = pda_subsampled[pda_subsampled['userID'].isin(pda_subsampled['userID'].value_counts()[pda_subsampled['userID'].value_counts()>10].index)]
print("\nShapes after sub-sampling:")
print(ml_subsampled.shape)
print(pda_subsampled.shape)

# Create the training datasets using Surprise's reader class
reader = Reader(rating_scale=(1,5)) # We have ratings from 1 to 5 so we create the rating scale

# Load the data from the dataframes
movielens_dataset = Dataset.load_from_df(movielens_df.iloc[:,0:3], reader)
pda_dataset = Dataset.load_from_df(pda_df.iloc[:,0:3], reader)

# Build full trainsets to print out the data loaded above
mls_train = movielens_dataset.build_full_trainset()
pda_train = pda_dataset.build_full_trainset()

# Print out some basic information about the datasets
print("\n Some general information on the training sets we will be using:")
print("1) Number of items in each dataset", " ML100k:", mls_train.n_items, "PDA:", pda_train.n_items)
print("2) Number of users in each dataset", " ML100k:", mls_train.n_users, "PDA:", pda_train.n_users)
print("3) Number of ratings in each dataset", " ML100k:", mls_train.n_ratings, "PDA:", pda_train.n_ratings)
print("4) Mean rating", " ML100k:", mls_train.global_mean, "PDA:", pda_train.global_mean)

ML100K
    userID  itemID  rating  timestamp
0     196     242       3  881250949
1     186     302       3  891717742
2      22     377       1  878887116
3     244      51       2  880606923
4     166     346       1  886397596



PDA2018
    userID  itemID  rating  timeStamp
0       5     648       5  978297876
1       5    1394       5  978298237
2       5    3534       5  978297149
3       5     104       4  978298558
4       5    2735       5  978297919

 Some general information on the training sets we will be using:
1) Number of items in each dataset  ML100k: 1682 PDA: 1824
2) Number of users in each dataset  ML100k: 943 PDA: 5690
3) Number of ratings in each dataset  ML100k: 100000 PDA: 470711
4) Mean rating  ML100k: 3.52986 PDA: 3.638361967321775


## Surprise experiment parameters and variables


### Define auxilliary functions to get Top-N and then calculate precision and recall in Surprise + more

In [3]:
# Check that we have ratings in the train set for all the users in the test set
def find_items_not_in_trainset(trainset, testset):
    items_not_in_train = []
    for _,itemId, _ in testset:    
        if itemId not in trainset.ir.keys():
            items_not_in_train.append(itemId)
            
    return items_not_in_train

        
def user_seen_items(userId):
    return [train_itemId for train_itemId, rating in mls_train.ur[userId]]

In [4]:
def GetTopN(predictions, n, minimumRating, criterion):
    topN = defaultdict(list)
    
    for index, row in predictions.iterrows():
        if (row[criterion] >= minimumRating):
            topN[int(row.uid)].append((int(row.iid), row[criterion]))

    for userID, ratings in topN.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        topN[int(userID)] = ratings[:n]

    return topN

In [5]:
def precision_recall_at_k(predictions, k=5, threshold=3.5):
    top_n_recoms_est = GetTopN(predictions_df, n=k, minimumRating=threshold, criterion="est")
    top_n_recoms_real = GetTopN(predictions_df, n=k, minimumRating=threshold, criterion="r_ui")
    above_threshold = predictions_df[predictions_df.r_ui >= threshold]

    precisions = {}
    recalls = {}

    for uid, est_topn in top_n_recoms_est.items():
        # Get items the user has already rated
        already_seen = user_seen_items(uid)
        # Get relevant items for the user
        n_rel_for_user = len(above_threshold[above_threshold.uid == uid])        
        tp = 0
        # Penalize the scores if:
        # - The item we are recommending was never seen in the training set (how could we recommend what we don't know?)
        # - The user has already rated this item: It's not a good recommendation since the user already knows it/has seen it
        for est_itemId, _ in est_topn:
            if(est_itemId in items_not_in_train or est_itemId in already_seen):
                tp += 0
            else:
                for real_itemId, _ in top_n_recoms_real[uid]:
                    if (est_itemId == real_itemId):
                        tp +=1
        
        precisions[uid] = tp/k
        recalls[uid] = tp/n_rel_for_user if n_rel_for_user != 0 else 0

    return precisions, recalls

### Run 5-fold cross validation

In [6]:
""" EXPERIMENT VARIABLES"""
# List that will contain the results of all experiments
results_table = []
kf = KFold(n_splits=5) # define number of k splits for cross validation using Surprise KFold

# Algorithms we will be using in this section
algorithms = {
    "UserKNN": KNNWithMeans(k=60, sim_options={'name': 'pearson_baseline', 'shrinkage': 25, 'user_based': True}),
    "ItemKNN": KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'shrinkage': 25, 'user_based': False}),
}
# Datasets
datasets = {
    "ML100": movielens_dataset,
    "PDA2018": pda_dataset
}

In [7]:
for dataset in datasets.keys():
    for algorithm in algorithms.keys():
        print("Running 5-fold cross validation with", algorithm, "on", dataset, "dataset ...")
        
        # Run cross validation
        best_prec_5 = 0
        best_prec_10 = 0
        best_rec_5 = 0
        best_rec_10 = 0
        fold_nr = 0
        
        for trainset, testset in kf.split(movielens_dataset):
            print("Fold  number", fold_nr)
            algorithms[algorithm].fit(trainset)
            predictions = algorithms[algorithm].test(testset)
            predictions_df = pd.DataFrame(predictions)
            predictions_df = predictions_df.iloc[:,:-1]
            items_not_in_trainset = find_items_not_in_trainset(trainset, testset)
            pres_at_5, recalls_at_5 = precision_recall_at_k(predictions_df, items_not_in_trainset, k=5)
            pres_at_10, recalls_at_10 = precision_recall_at_k(predictions_df, items_not_in_trainset, k=10)
            avg_pre_5 = np.array([pres_at_5[k] for k in pres_at_5.keys()]).mean()
            avg_rec_5 = np.array([recalls_at_5[k] for k in recalls_at_5.keys()]).mean()
            avg_pre_10 = np.array([pres_at_10[k] for k in pres_at_10.keys()]).mean()
            avg_rec_10 = np.array([recalls_at_10[k] for k in recalls_at_10.keys()]).mean()
            if (avg_pre_5 > best_prec_5):
                best_prec_5 = avg_pre_5
            if (avg_pre_10 > best_prec_10):
                best_prec_10 = avg_pre_10
            if (avg_rec_5 > best_rec_5):
                best_rec_5 = avg_rec_5
            if (avg_rec_10 > best_rec_10):
                best_rec_10 = avg_rec_10
            fold_nr += 1
        
        new_line = [dataset+"-"+algorithm, best_prec_5, best_prec_10, best_rec_5, best_rec_10]
        results_table.append(new_line)
        print()

Running 5-fold cross validation with UserKNN on ML100 dataset ...
Fold  number 0
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Fold  number 1
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Fold  number 2
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Fold  number 3
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Fold  number 4
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Running 5-fold cross validation with ItemKNN on ML100 dataset ...
Fold  number 0
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Fold  number 1
Estimating biases using als...
Computing the pearson_baseline

# Surprise Results

In [9]:
# Export the results to a csv file
# results_df = pd.DataFrame(results_table, columns=["Recommender", "RMSE", "MAE"])
# results_df.to_csv("../data/rating_prediction_results.csv")

# Part 2: Using Cornac

The implementations of BPR and NCF are done by using the Cornac package, which is similiar to Surprise, but contains much more algorithms and metrics. 
- NCF (Neural Collaborative Filtering) is a generalization of the matrix factorization problem using a multilayer perceptron.

- BPR (Bayesian Personalised Ranking) is esentially a Matrix Factorization algorithm which is optimized with a Bayes Criterion (BPR-OPT) in order to make the recommendation list ranking as personalized to the user (hence as "correct") as possible. 

In [10]:
import numpy as np
import pandas as pd
import cornac
from cornac.eval_methods import CrossValidation, RatioSplit
from cornac.data import Reader
from cornac.data import Dataset
from cornac.hyperopt import Discrete, GridSearch
from tabulate import tabulate

## Read and load the data using Cornac

In [23]:
# Init cornac reader object
reader = Reader() # this binarises the data (turns it into implicit feedback)

# Read both the datasets from the files using cornac
movielens_data = reader.read(fpath="../data/u.data", sep="\t")
pda_data = reader.read(fpath="../data/train-PDA2018.csv", sep=",", skip_lines=1)
print("Movielens")
print(movielens_data[:5])
print()
print("PDA")
print(pda_data[:5])

Movielens
[('196', '242', 3.0), ('186', '302', 3.0), ('22', '377', 1.0), ('244', '51', 2.0), ('166', '346', 1.0)]

PDA
[('5', '648', 5.0), ('5', '1394', 5.0), ('5', '3534', 5.0), ('5', '104', 4.0), ('5', '2735', 5.0)]


## Define models and metrics

In [36]:
# First of let's define the NCF model using cornac. Cornac gives us the possibility to tweak a lot of hyperparameters
# We'll used the Pre-Trained NeuMF model, since it performs better as shown in https://arxiv.org/pdf/1708.05031.pdf

gmf = cornac.models.GMF(
    num_factors=8,
    num_epochs=10,
    learner="adam",
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
mlp = cornac.models.MLP(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
ncf_model = cornac.models.NeuMF(
    name="NeuMF_pretrained",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    num_factors=gmf.num_factors,
    layers=mlp.layers,
    act_fn=mlp.act_fn,
).pretrain(gmf, mlp)


bpr_model = cornac.models.BPR(
    k=10,
    learning_rate=0.05,
    lambda_reg=0.05
)
# Metrics
pre_5 = cornac.metrics.Precision(k=5)
pre_10 = cornac.metrics.Precision(k=10)
rec_5 = cornac.metrics.Recall(k=5)
rec_10 = cornac.metrics.Recall(k=10)
ndcg_5 = ndcg_5 = cornac.metrics.NDCG(k=5)
ndcg_10 = cornac.metrics.NDCG(k=10)
auc = cornac.metrics.AUC()

In [37]:
""" EXPERIMENT VARIABLES"""
# List that will contain the RMSE and MAE results
cv_n_folds = 5 # define number of k splits for cross validation
rating_threshold = 3 # This parameter is the threshold used for ranking metrics

# Algorithms we will be using in this section
cornac_algorithms = {
    "NCF": ncf_model, 
    "BPR": bpr_model, 
}
# Datasets
datasets = {
    "ML100": movielens_data,
    "PDA2018": pda_data
}

## 5-Fold Cross Validation

In [39]:
for dataset in datasets.keys():
    for algorithm in algorithms.keys():
        print("Running 5-fold cross validation with", algorithm, "on", dataset, "dataset ...\n\n")
        # Define Cornac cross validation object
        cv = CrossValidation(
            data=datasets[dataset],
            n_folds=cv_n_folds,
            rating_threshold=rating_threshold, # This parameter is the threshold used for ranking metrics
            seed = 0,
            verbose=True
        )
        # Define Cornac experiment (put everything together)
        experiment = cornac.Experiment(
            eval_method=cv,
            models=[algorithms[algorithm]],    
            metrics=[pre_5, pre_10, rec_5, rec_10, auc],
        )
        experiment.run()
        for entry in experiment.result:
            results_dict = entry[0].metric_avg_results
            new_line = [dataset+"-"+algorithm, results_dict['Precision@5'], results_dict['Precision@10'], results_dict['Recall@5'], results_dict['Recall@5']]
            results_table.append(new_line)

Running 5-fold cross validation with NCF on ML100 dataset ...


rating_threshold = 3.0
exclude_unknowns = True
Fold: 1
---
Training data:
Number of users = 943
Number of items = 1648
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1382
Number of ratings = 19966
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 943
Number of items = 1382
Number of ratings = 19966
---
Total users = 943
Total items = 1648

[NeuMF] Training started!




[NeuMF] Evaluation started!



Fold: 2
---
Training data:
Number of users = 943
Number of items = 1652
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 942
Number of items = 1371
Number of ratings = 19967
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 942
Number of items = 1371
Number of ratings = 19967
---
Total users = 943
Total items = 1652

[NeuMF] Training started!




[NeuMF] Evaluation started!



Fold: 3
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 940
Number of items = 1390
Number of ratings = 19965
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 940
Number of items = 1390
Number of ratings = 19965
---
Total users = 943
Total items = 1651

[NeuMF] Training started!


...


Fold: 2
---
Training data:
Number of users = 5685
Number of items = 1821
Number of ratings = 376569
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5294
Number of items = 1791
Number of ratings = 94127
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5294
Number of items = 1791
Number of ratings = 94127
---
Total users = 5685
Total items = 1821

[BPR] Training started!

[BPR] Evaluation started!



Fold: 3
---
Training data:
Number of users = 5684
Number of items = 1823
Number of ratings = 376568
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5328
Number of items = 1798
Number of ratings = 94129
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5328
Number of items = 1798
Number of ratings = 94129
---
Total users = 5684
Total items = 1823

[BPR] Training started!

[BPR] Evaluation started!



Fold: 4
---
Training data:
Number of users = 5684
Number of items = 1822
Number of ratings = 376569
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5303
Number of items = 1787
Number of ratings = 94125
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5303
Number of items = 1787
Number of ratings = 94125
---
Total users = 5684
Total items = 1822

[BPR] Training started!

[BPR] Evaluation started!



Fold: 5
---
Training data:
Number of users = 5682
Number of items = 1820
Number of ratings = 376569
Max rating = 5.0
Min rating = 1.0
Global mean = 3.6
---
Test data:
Number of users = 5281
Number of items = 1799
Number of ratings = 94121
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 5281
Number of items = 1799
Number of ratings = 94121
---
Total users = 5682
Total items = 1820

[BPR] Training started!

[BPR] Evaluation started!




TEST:
...
[BPR]
       | Precision@10 | Precision@5 | Recall@10 | Recall@5 | Train (s) | Test (s)
------ + ------------ + ----------- + --------- + -------- + --------- + --------
Fold 0 |       0.0851 |      0.0873 |    0.0930 |   0.0496 |    3.3262 |   4.5780
Fold 1 |       0.0874 |      0.0939 |    0.0921 |   0.0513 |    3.4355 |   4.5098
Fold 2 |       0.0879 |      0.0933 |    0.0900 |   0.0494 |    3.3357 |   4.5170
Fold 3 |       0.0842 |      0.0896 |    0.0882 |   0.0485 |    3.3260 |   4.5395
Fold 4 |       0.0873 |      0.0944 |    0.0915 |   0.0519 |    4.4721 |   5.7157
------ + ------------ + ----------- + --------- + -------- + --------- + --------
Mean   |       0.0864 |      0.0917 |    0.0910 |   0.0501 |    3.5791 |   4.7720
Std    |       0.0014 |      0.0028 |    0.0017 |   0.0013 |    0.4484 |   0.4725



In [40]:
# Display results of running the algorithms
results_table_headers = ["Recommender", "Pre@5", "Pre@10", "Rec@5", "Rec@10"]
print(tabulate(results_table, results_table_headers, tablefmt="pipe"))

| Recommender   |     Pre@5 |    Pre@10 |     Rec@5 |    Rec@10 |
|:--------------|----------:|----------:|----------:|----------:|
| ML100-NCF     | 0.109671  | 0.104782  | 0.0581365 | 0.0581365 |
| ML100-BPR     | 0.123911  | 0.115728  | 0.0652518 | 0.0652518 |
| PDA2018-NCF   | 0.0840333 | 0.0801551 | 0.0461607 | 0.0461607 |
| PDA2018-BPR   | 0.0872872 | 0.0851495 | 0.0496115 | 0.0496115 |


In [41]:
results_table

[['ML100-NCF',
  0.10967056323060649,
  0.1047821466524971,
  0.0581365403103877,
  0.0581365403103877],
 ['ML100-BPR',
  0.12391073326248765,
  0.11572794899043558,
  0.06525178411354834,
  0.06525178411354834],
 ['PDA2018-NCF',
  0.08403329549753837,
  0.08015512674990725,
  0.04616065078662213,
  0.04616065078662213],
 ['PDA2018-BPR',
  0.08728717366628583,
  0.08514945138100856,
  0.04961145771911425,
  0.04961145771911425]]